## Environment setup

In [ ]:
# !pip install pubchempy
# !sudo apt install swig libopenbabel-dev python3-openbabel
# !pip install --upgrade pip
# !pip install openbabel 

## Download SDF file according to list of metabolites

In [3]:
import pubchempy as pcp
folder = '/media/favreau/Documents/medias/pdb/metabolites'

In [4]:
def download_sdf(name):
    cs = pcp.get_compounds(name, 'name')
    for c in cs:
        cid = c.cid
        pcp.download(
            outformat='SDF', path='%s/%s_%d.sdf' % (folder, name, cid),
            identifier=cid, overwrite=True)

In [5]:
metabolites = open('Metabolites.txt', 'r')
lines = metabolites.readlines()
for line in lines:
    values = line.split('_')
    download_sdf(values[0])
metabolites.close()

## Convert SDF files to PDB

In [ ]:
import glob
import openbabel as ob

sdf_files = glob.glob('%s/*.sdf' % folder)

for sdf_file in sdf_files:
    try:
        conv = ob.OBConversion()
        conv.SetInAndOutFormats("sdf","pdb")
        mol= ob.OBMol()
        conv.ReadFile(mol, sdf_file)
        conv.WriteFile(mol, sdf_file[:-4]+".pdb")
    except:
        raise

## Visualize in BioExplorer

In [ ]:
from bioexplorer import BioExplorer, Protein, Vector3, Volume
be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset()

scene_size = 300

In [ ]:
import os

pdb_files = glob.glob('%s/*.pdb' % folder)
count = 0
nb_proteins = 100 # Use proper concentration here according to timestamp!!
for pdb_file in pdb_files:
    name=os.path.basename(pdb_file)
    metabolite = Protein(
        sources=[pdb_file], load_non_polymer_chemicals=True, 
        occurences=nb_proteins)
    volume = Volume(
        random_rotation_seed=count, random_rotation_stength=1.0,
        name=name, size=scene_size, protein=metabolite)
    status = be.add_volume(
        volume=volume, random_seed=count,
        representation=be.REPRESENTATION_ATOMS_AND_STICKS,
        atom_radius_multiplier=1.0)
    count += 1